<a href="https://colab.research.google.com/github/yalopez84/Goog-Negative-Sampling/blob/master/Basic_Preprocessing_FB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Basic Form Preprocessing**

Basic Form Preprocessing means to transform any KG dataset to the Basic Form for kg completion algorithms. Basic Form was drawn from KG-BERT and extended with the relation_domain_range.tsv file. This file contains the relation domain and range used by the Good Negative Sampling generation strategy. Basic Form files are:

1.   train.tsv
2.   dev.tsv
3.   test.tsv
4.   entities.tsv
5.   relations.tsv
6.   entity2text.tsv
7.   relation2text.tsv
8.   relation_dom_ran.tsv


In [9]:
import os
from tqdm import tqdm, trange
import csv

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
data_dir="/content/drive/MyDrive/NegativeStrategies/GoodNegativeSampling/FB13/"
os.chdir(data_dir)

In [12]:
class DataProcessor(object):
    def get_train_examples(self, data_dir):
        raise NotImplementedError()
    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        with open(input_file, "r", encoding="utf-8") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []            
            for line in reader:
                lines.append(line)  
            return lines  

In [ ]:
class KGProcessor(DataProcessor):
    def __init__(self):
        self.labels = set()        
    

**createFBReduced method** is for creating reduced files of train.tsv and test.tsv

In [13]:
def createFBReduced(test_start,test_end,test_step,train_start,train_end,train_step):
    #To reduce FB dataset 
    #train.tsv has 316219 triples, train_reduced.tsv has a subset of 11082 triples
    #test.tsv has 47464, test_reduced.tsv has a sub set of 2253 triples  

    processor = KGProcessor()
    test_lines=processor._read_tsv(os.path.join(data_dir, "test.tsv"))
    train_lines=processor._read_tsv(os.path.join(data_dir, "train.tsv")) 

    triples_train_reduced=processor._read_tsv(os.path.join(data_dir, "train_reduced.tsv"))
    triples_test_reduced=processor._read_tsv(os.path.join(data_dir, "test_reduced.tsv"))
    flag=False

    for test_line in tqdm(test_lines[test_start:test_end:test_step]):
        tests_str_set = set([' '.join(line) for line in triples_test_reduced])
        flag=False
        count=0
        for train_line in train_lines[train_start:train_end:train_step]:
            trains_str_set = set([' '.join(line) for line in triples_train_reduced])
            if test_line[0] in train_line or test_line[1] in train_line or test_line[2] in train_line:
                if ' '.join(train_line) not in trains_str_set:
                    triples_train_reduced.append(train_line) 
                    flag=True
                    count=count+1
                    if count>=2:
                        break
        if flag==True:
            if ' '.join(test_line) not in tests_str_set:
                triples_test_reduced.append(test_line) 

    print("train_reduced", len(triples_train_reduced),"\n")
    print("train_reduced", triples_train_reduced,"\n")

    print("test_reduced", len(triples_test_reduced),"\n")
    print("test_reduced", triples_test_reduced,"\n")

    positivos=[triple for triple in triples_test_reduced if triple[3]=="1"]
    negativos=[triple for triple in triples_test_reduced if triple[3]=="-1"]
    print("positivos", len(positivos),"\n")
    print("negativos", len(negativos),"\n")

    with open(os.path.join(data_dir, "train_reduced.tsv"), "w") as writer:
        for triple_ in triples_train_reduced:
            writer.write('\t'.join(triple_)+"\n")

    with open(os.path.join(data_dir, "test_reduced.tsv"), "w") as writer:
        for triple_ in triples_test_reduced:
            writer.write('\t'.join(triple_)+"\n")

#createFBReduced(test_start=0,test_end=0,test_step=1,train_start=0,train_end=0,train_step=1)